In [1]:
import pandas as pd
import datetime
import re
import os
import warnings
import numpy as np
warnings.filterwarnings('ignore')

# Carga de Bases de datos estandarizadas

## Definición de Bases de datos y rutas

In [2]:
path = os.path.join(os.path.expanduser('~'), r'Medtronic PLC\Approvals and Databases SSC - Documents\Databases')
countries = {
    'BO': 'Bolivia',
    'CO': 'Colombia',
    'CR': 'Costa Rica',
    'EC': 'Ecuador',
    'SV': 'El Salvador',
    'GT': 'Guatemala',
    'MX': 'Mexico',
    'PE': 'Perú',
    'VE': 'Venezuela',
    'PY': 'Paraguay',
}
# Ruta para encontrar las BBDD
path_db = {
    'BO': r'\Bolivia\MDT Bolivia DB.xlsm',
    'CO': r'\Colombia\MDT Colombia DB.xlsm',
    'CR': r'\Costa Rica\MDT Costa Rica DB.xlsm',
    'EC': r'\Ecuador\MDT Ecuador DB.xlsm', 
    'PE': r'\Perú\MDT Perú DB.xlsm', 
    'SV': r'\El Salvador\MDT El Salvador DB.xlsm', 
    'MX': r'\Mexico\MDT Mexico DB.xlsm',
    'GT': r'\Guatemala\MDT Guatemala DB.xlsm',
    'VE': r'\Venezuela\MDT Venezuela DB.xlsm',
    'PY': r'\Paraguay\MDT Paraguay DB.xlsm',
}

In [3]:
df = pd.DataFrame(columns= ['Country','REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING SITE','LICENSE HOLDER'])
for country in countries.keys():
    db_path = path+path_db[country]
    print(db_path)
    temporal =pd.read_excel(db_path,sheet_name = 'ACTIVE CODES',usecols= ['REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING SITE','LICENSE HOLDER'],converters={'CFN':str,'REGISTRATION NUMBER':str},
                            date_parser = ['EXPIRATION DATE'])
    temporal['Country'] = country
    df = pd.concat([df,temporal],ignore_index=True)

C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Bolivia\MDT Bolivia DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Colombia\MDT Colombia DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Costa Rica\MDT Costa Rica DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Ecuador\MDT Ecuador DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\El Salvador\MDT El Salvador DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Guatemala\MDT Guatemala DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Mexico\MDT Mexico DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Perú\MDT Perú DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Venezuela\MDT Ven

## Carga de Uruguay como excepción

In [4]:
db_path = path + r'\Uruguay\MDT Uruguay DB.xlsm'
temporal =pd.read_excel(db_path,sheet_name = 'ACTIVE CODES',usecols= ['REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','OU','MANUFACTURING SITE','LICENSE HOLDER'],converters={'CFN':str,'REGISTRATION NUMBER':str},
                            date_parser = ['EXPIRATION DATE'])
temporal['Country'] = 'UY'
temporal['CFN DESCRIPTION'] = 'No disponible en BD'

### Adheción al DataFrame general

In [5]:
df = pd.concat([df,temporal],ignore_index=True)

# Carga de Base de datos Brasil

## Carga inicial de los datos de Brasil

In [6]:
brand_code = {
    'MDT':"Medtronic",
    'COV': "Covidien"
}
path_db = {
    'COV': r'\Brasil\Piloto Oficial_COV_2020.05.22.xlsm',
    'MDT': r'\Brasil\Piloto Oficial_MDT_2020.06.08.xlsm'
}

In [7]:
df_brasil = pd.DataFrame(columns= ['Country','Registro ANVISA','Nome do Registro','Status do Registro','Data de Vencimento do Registro ','Código','Descrição do Código','BU','Fabricante Físico (Real)','Detentor do Registro'])
for bc in brand_code.keys():
    db_path = path+path_db[bc]
    print(db_path)
    temporal =pd.read_excel(db_path,sheet_name = 'Banco de Dados',
                            usecols= ['Registro ANVISA','Nome do Registro','Status do Registro','Data de Vencimento do Registro ','Código','Descrição do Código','BU','Fabricante Físico (Real)','Detentor do Registro'],converters={'Código':str,'Registro ANVISA':str},
                            date_parser = ['Data de Vencimento do Registro '])
    temporal['Country'] = 'BR'
    df_brasil = pd.concat([df_brasil,temporal],ignore_index=True)
df_brasil = df_brasil.rename(columns={'Código':'CFN','BU':'OU','Registro ANVISA':'REGISTRATION NUMBER','Data de Vencimento do Registro ':'EXPIRATION DATE',
                               'Nome do Registro':'REGISTRATION NAME', 'Descrição do Código':'CFN DESCRIPTION','Status do Registro':'STATUS','Fabricante Físico (Real)':'MANUFACTURING SITE',
                               'Detentor do Registro':'LICENSE HOLDER'})

C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Brasil\Piloto Oficial_MDT_2020.06.08.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Brasil\Piloto Oficial_COV_2020.05.22.xlsm


## Pre Procesamiento y adheción al DataFrame principal

In [8]:
df_brasil = df_brasil[~df_brasil['STATUS'].isin(['Cancelado','OBSOLETO','obsoleto','Obsoleto','\\','Vencido'])]
df = pd.concat([df,df_brasil],ignore_index=True)

# Carga de las Bases de Datos de Argentina

In [9]:
brand_code = {
    'MDT':"Medtronic",
    'COV': "Covidien"
}
path_db = {
    'COV': r'\Argentina\COV Argentina DB.xlsm',
    'MDT': r'\Argentina\MDT Argentina DB.xlsm'
}

## Carga inicial de los Datos

In [10]:
df_AR = pd.DataFrame(columns = ['Country','REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING NAME','MANUFACTURING ADDRESS','LICENSE HOLDER'] )
for bc in brand_code.keys():
    db_path = path+path_db[bc]
    print(db_path)
    temporal =pd.read_excel(db_path,sheet_name = 'ACTIVE CODES',usecols= ['REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING NAME','MANUFACTURING ADDRESS','LICENSE HOLDER'],
                            date_parser = ['EXPIRATION DATE'],converters = {'REGISTRTION NUMBER':str,'CFN':str,} )
    temporal['Country'] = 'AR'
    df_AR = pd.concat([df_AR,temporal],ignore_index=True)


C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Argentina\MDT Argentina DB.xlsm
C:\Users\rodrij545\Medtronic PLC\Approvals and Databases SSC - Documents\Databases\Argentina\COV Argentina DB.xlsm


## Pre Procesado de los datos

### Concatenación de manufacturer

In [11]:
def cut_values(row,column = 'MANUFACTURING ADDRESS'):
    var = str(row[column])
    if '\n' in var:
        var = var.split('\n')
        var1 = [name.strip() for name in var]
        return var
    else:
        return var


In [12]:
df_AR['CUT ADDRESS'] = df_AR.apply(cut_values,axis = 1)
df_AR['CUT NAME'] = df_AR.apply(cut_values,axis = 1,column = 'MANUFACTURING NAME')

In [13]:
def trim_column(row,column = 'REGISTRATION NUMBER'):
    a = str(row[column])
    a = a.strip()
    return a


In [14]:
def paste_problem(row,name ='CUT ADDRESS',address =  'CUT ADDRESS'):
    a = row[name]
    b = row[address]
    if type(a) is list:
        junto = ''
        for nombre,dir in zip(a,b):
            junto += nombre + ' ' + dir + '\n'
        return junto
    else:
        junto = a + ' ' + b + '\n'
        return junto

    

In [15]:
df_AR['MANUFACTURING SITE'] = df_AR.apply(paste_problem,axis=1)

## Adherir a el dataframe general los datos de Argentina

In [16]:
temporal = pd.DataFrame(columns=['REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','Country','MANUFACTURING SITE','LICENSE HOLDER'])
for column in temporal.columns:
    temporal[column] = df_AR[column]
df = pd.concat([df,temporal],ignore_index=True)

# Carga de la base de datos de Honduras

## Hoja Principal

### Carga inicial de datos

In [17]:
db_path = path + r'\Honduras\MDT-MITG Base de datos Honduras.xlsx'
honduras = pd.read_excel(db_path,usecols='A:L',converters = {'Registration number':str,'CFN':str})
honduras = honduras.rename(columns={'BU':'OU','Descripción':'CFN DESCRIPTION','Approval date \n(dia-Mes-YY)':'APPROVAL DATE','Expire date \n(dia-Mes-YY)':'EXPIRATION DATE',
                                    'Distribuidor':'DISTRIBUTOR','Product name':'REGISTRATION NAME','Manufacturing site 2 (If apply)':'Manufacturing site 2',
                                    'Registration number':'REGISTRATION NUMBER'},
                                    
                                    )
honduras['Country'] = 'HN'

### Pre Procesamiento de Honduras

In [18]:
def concatMfg(row,colum1 = 'Manufacturing site 1',colum2 = 'Manufacturing site 2'):
    mfg1 = str(row[colum1])
    mfg2 = str(row[colum2])
    mfg = mfg1 + '\n' + mfg2
    return mfg

### Columnas faltantes

In [19]:
honduras['MANUFACTURING SITE'] = honduras.apply(concatMfg,axis=1)
honduras['STATUS'] = 'No disponible en BD'
honduras['LICENSE HOLDER'] = 'No disponible en BD'

### Adherir a el dataframe general los datos de honduras (principal)

In [20]:
temporal = pd.DataFrame(columns=['Country','REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING SITE','LICENSE HOLDER'])
for column in temporal.columns:
    temporal[column] = honduras[column]
df = pd.concat([df,temporal],ignore_index=True)

## Carga base de datos para MITG Honduras

### Carga MITG

In [21]:
db_path = path + r'\Honduras\MDT-MITG Base de datos Honduras.xlsx'
honduras = pd.read_excel(db_path,sheet_name='MITG Local',converters = {'Nº LICENSE':str,'CODES':str})
honduras = honduras.rename(columns={'CODES':'CFN','LÍNEA':'OU','Nº LICENSE':'REGISTRATION NUMBER','DESCRPTION OF THE REFERENCE CODE':'CFN DESCRIPTION',
                                    'ADDRESS':'MANUFACTURING SITE','DESCRIPCION OF APPROVAL':'REGISTRATION NAME','EXPIRATION \nDAY':'EXPIRATION DATE',
                                    'APPROVAL \nDATE':'APPROVAL DATE'}
                          )
honduras['Country'] = 'HN'

### Columnas faltantes

In [22]:
honduras['STATUS'] = 'No disponible en BD'
honduras['LICENSE HOLDER'] = 'No disponible en BD'

### Adherir a el dataframe general los datos de honduras (MITG)

In [23]:
temporal = pd.DataFrame(columns=['Country','REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING SITE','LICENSE HOLDER'])
for column in temporal.columns:
    temporal[column] = honduras[column]
df = pd.concat([df,temporal],ignore_index=True)

# Carga Base de Datos Republica Dominicana

## Carga inicial de la Base de datos

In [24]:
db_path = path + r'\República Dominicana\MITG Base de datos Republica Dominicana.xlsx'
RepDo = pd.read_excel(db_path,usecols='A:M',converters = {'REGISTRO SANITARIO No.':str,'REFERENCIA':str})

RepDo = RepDo.rename(columns={'REFERENCIA':'CFN','REGISTRO SANITARIO No.':'REGISTRATION NUMBER','TITULAR':'LICENSE HOLDER','FABRICADO POR':'MANUFACTURING SITE',
                              'BU':'OU','VIGENCIA DEL REGISTRO SANITARIO (dd/mm/aaaa)':'EXPIRATION DATE','DESCRIPCIÓN DE REFERENCIA':'CFN DESCRIPTION',
                              'DENOMINACION DEL PRODUCTO SEGÚN REGISTRO SANITARIO':'REGISTRATION NAME'})

RepDo['STATUS'] =  'No disponible en BD'
RepDo['Country'] = 'DO'
RepDo.columns

Index(['CFN', 'CFN DESCRIPTION', 'OU', 'REGISTRATION NUMBER',
       'EXPIRATION DATE', 'REGISTRATION NAME', 'LICENSE HOLDER',
       'MANUFACTURING SITE', 'PAIS FABRICANTE', 'VIDA ÚTIL', 'RESOLUCIÓN',
       'FECHA DE EXPEDICIÓN', 'COMENTARIOS', 'STATUS', 'Country'],
      dtype='object')

## Adherir republica dominicana al dataframe general

In [25]:
temporal = pd.DataFrame(columns=['Country','REGISTRATION NUMBER','REGISTRATION NAME','STATUS','EXPIRATION DATE','CFN','CFN DESCRIPTION','OU','MANUFACTURING SITE','LICENSE HOLDER'])
for column in temporal.columns:
    temporal[column] = RepDo[column]
df = pd.concat([df,temporal],ignore_index=True)

# Trimear datos del dataframe General

In [26]:
for col in ['CFN', 'REGISTRATION NUMBER','REGISTRATION NAME','OU']:
    df[col] = df.apply(trim_column, axis = 1, column = col)
df = df.sort_values(by = ['Country'])

# Trabajo con el Submission Plan

## Cargar Submission Plan para cruces de información

In [27]:
df_plan = pd.read_excel('Submission Plan - Full Report.xlsx',usecols=['Id','License Number','SubOU','Status','Submission Type','Expected Submission Date','Expected Approval Date','Approval Date','Country'],
                        converters = {'License Number':str})


## Pre procesamiento de datos

In [28]:
df_plan=df_plan.rename(columns={'License Number':'REGISTRATION NUMBER','Status':'Status Submission Plan'})

In [29]:
df_plan['REGISTRATION NUMBER'] = df_plan.apply(trim_column,axis=1,column='REGISTRATION NUMBER')

# Tracker

## Tracker por OU

In [37]:
df_ou = df[df['OU'].isin(['SI(LA)','SI (LA)','SI'])]
df_registration = df_ou.drop('CFN',axis = 1)
df_registration = df_registration.drop_duplicates(subset=['REGISTRATION NUMBER'])


In [38]:
repo = pd.merge(df_plan,df_registration, how='inner',on='REGISTRATION NUMBER')
repo = repo[~repo['Status Submission Plan'].isin(['APPROVED'])]

In [40]:
with pd.ExcelWriter('tracker Laparoscopia.xlsx') as writer1:
    df_ou.to_excel(writer1, sheet_name = 'Listado con CFNs', index = False)
    df_registration.to_excel(writer1, sheet_name = 'Listado de Regsitros', index = False)
    repo.to_excel(writer1, sheet_name  = ' Busqueda en Submission Plan',index = False)
    

## Tracker por Pais

In [71]:
df_country = df[df['Country'] == 'BO']
df_registration = df_country.drop('CFN',axis=1)
df_registration = df_registration.drop_duplicates(subset=['REGISTRATION NUMBER'])
repo = pd.merge(df_plan,df_registration, how='inner',on='REGISTRATION NUMBER')

In [39]:
full = df_plan[df_plan['Country'] == 'BO - Bolivia']

In [79]:
with pd.ExcelWriter('tracker Bolivia.xlsx') as writer1:
    df_country.to_excel(writer1, sheet_name = 'Lsitado con CFNs', index = False)
    df_registration.to_excel(writer1, sheet_name = 'Listado de Regsitros', index = False)
    repo.to_excel(writer1, sheet_name  = ' Busqueda en Submission Plan',index = False)
    full.to_excel(writer1, sheet_name  = ' Full Submmission Plan',index = False)

## Cosolidado de paises

In [ ]:
df.to_excel('Consolidado general.xlsx',index = False)

# Tracker de OU antiguas

In [30]:
listado = """CAS
CORO
CRM
CS
CSF
CV
DBS
DIAB
ENT
GIH
INT
KH
MCS
MPSS
MSB
NAV
NIM
NV
PAIN
PH
PM
PVH
RCS
RDN
RI
SHA
SI(A&I)
SI(EMID)
SI(HW,VS&ES)
SI(LA)
SI(OS&SP)
SI(ST)
SI(SY,SO&MS)
SR
TS"""


In [36]:
listado = listado.split('\n')
listado1 = [var.strip() for var in listado]

In [37]:
df_ou = df[~df['OU'].isin(listado1)]
# df_ou.to_excel('CFNs con SubOU antiguas.xlsx',index = False)

## Por Lsita de CFNs

In [31]:
lista = pd.read_excel('cfn_list.xlsx',converters = {'CFN':str})

In [32]:
lista1 = [cfn.strip() for cfn in lista['CFN'] ]

In [33]:
recuento = pd.DataFrame(columns = ['CFN','SubOUs','Country'])
for cfn in lista1:
    aux = pd.DataFrame(columns = ['CFN','SubOUs'])
    temp = df[df['CFN'] == cfn]
    pos = [op for op in temp['OU']]
    countries = [op for op in temp['Country']]
    
    aux['CFN'] = [cfn]
    aux['SubOUs'] = [pos]
    aux['Country'] = [countries]
    recuento = pd.concat([recuento,aux])


In [85]:
compare = pd.DataFrame(columns = ['CFN','SubOUs','Country'])
for cfn in set(df['CFN']):
    aux = pd.DataFrame(columns = ['CFN','SubOUs','Country'])
    temp = df[df['CFN'] == cfn]
    pos = [op for op in temp['OU']]
    countries = [op for op in temp['Country']]
    val = np.unique(np.array(pos))
    if  len(val)>1:
        aux['CFN'] = [cfn]
        aux['SubOUs'] = [pos]
        aux['Country'] = [countries]
        compare = pd.concat([compare,aux])
    else:
        continue

In [73]:
df_out2 = df[df['CFN'].isin(lista1)]


In [38]:
with pd.ExcelWriter('SubOU_review.xlsx') as writer1:
    df_ou.to_excel(writer1, sheet_name = 'Listado de CFNs con OU viejas', index = False)
    recuento.to_excel(writer1, sheet_name  = 'OU no detectadas comparadas con otros paises',index = False)
    compare.to_excel(writer1,sheet_name='Equivalencias por paises',index = False)

# Trabajo con potenciales critical comunication

In [30]:
critical = pd.read_excel('Expected Critical Communications Report.xlsx',converters={'License Number':str})
critical = critical[critical['Status'] == 'CANCELLED']

In [49]:
critical.columns

Index(['Product Name', 'Id', 'RAS Name', 'License Number', 'CFN',
       'Submission Type', 'License Expiration Date', 'Cluster', 'Country',
       'Therapy Group', 'SubOU', 'Status', 'Expected Approval Date',
       'RAC/RAN'],
      dtype='object')

In [31]:
listado = [var.strip() for var in critical['License Number']]
listado = set(listado)

In [51]:
df.columns

Index(['Country', 'REGISTRATION NUMBER', 'REGISTRATION NAME', 'STATUS',
       'EXPIRATION DATE', 'CFN', 'CFN DESCRIPTION', 'OU', 'MANUFACTURING SITE',
       'LICENSE HOLDER'],
      dtype='object')

In [32]:
df_critical = df[(df['REGISTRATION NUMBER'].isin(listado)) & (df['STATUS']!= 'Vigente, no se renovará')]